In [ ]:
import torch


# 步骤1：评估通道重要性（以BN层γ为例）
def compute_channel_importance(model):
    gamma_values = []
    for name, module in model.named_modules():
        if isinstance(module, nn.BatchNorm2d):
            gamma = module.weight.data.abs()  # γ绝对值作为重要性
            gamma_values.append(gamma)
    return torch.cat(gamma_values)

# 步骤2：生成通道掩码
def generate_prune_mask(gamma, prune_ratio=0.3):
    sorted_idx = torch.argsort(gamma)
    prune_num = int(len(gamma) * prune_ratio)
    mask = torch.ones_like(gamma)
    mask[sorted_idx[:prune_num]] = 0  # 剪除重要性最低的通道
    return mask

# 步骤3：应用剪枝（以Conv-BN结构为例）
def apply_channel_prune(conv, bn, mask):
    # 剪枝BN层
    bn.weight.data = bn.weight.data[mask == 1]
    bn.bias.data = bn.bias.data[mask == 1]
    bn.running_mean = bn.running_mean[mask == 1]
    bn.running_var = bn.running_var[mask == 1]
    
    # 剪枝卷积层输出通道
    conv.weight.data = conv.weight.data[mask == 1, :, :, :]
    if conv.bias is not None:
        conv.bias.data = conv.bias.data[mask == 1]
    
    # 调整下一层卷积输入通道
    next_conv = get_next_conv_layer(model, conv)
    if next_conv is not None:
        next_conv.weight.data = next_conv.weight.data[:, mask == 1, :, :]